In [1]:
import time
import pandas as pd
import numpy as np
import os
import os.path
from osgeo import gdal, ogr, osr
from scipy import misc, ndimage
gdal.UseExceptions()
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline
import urllib
import sys
sys.path.append("../Code")
import utils
import config
import shutil

In [ ]:
!conda install scipy --yes

In [22]:
key=config.api_token

In [24]:
dataset= pd.read_csv("../Data/Intermediate_files/hh_data_2011_cluster_minHH.csv")
output_image_dir='../Data/googleimage_matchinglsms_10percluster/'
#dataset= pd.read_csv("../Data/Intermediate_files/hh_village_coord_fcs.csv")

In [25]:
dataset.head()

,Unnamed: 0,gpsLatitude,gpsLongitude,cons,n,poor,n2,poor_majority,i,j
0,1,-1.404995,29.949390,2.316352,12,0.250000,12,0,25194,9169
1,3,-1.340851,29.768257,1.594851,11,0.545455,11,1,25172,9161
2,4,-1.255216,30.056820,3.852909,10,0.300000,10,0,25207,9151
3,5,-1.241430,29.961166,2.776385,8,0.500000,8,1,25195,9149
4,8,-1.166219,29.794294,2.971421,7,0.142857,7,0,25175,9140


In [26]:
raster_file = '../Data/Satellite/NightLight/F182013.v4c_web.stable_lights.avg_vis.tif'
x_size, top_left_x_coords, top_left_y_coords, centroid_x_coords, centroid_y_coords, bands_data = utils.read_raster(raster_file)

In [27]:
def max_100pixs(lon, lat):
    i,j=get_cell_idx(lon, lat, top_left_x_coords, top_left_y_coords)
    values=[]
    for a in range(-5,5):
        for b in range(-5,5):
            values.append(bands_data[j+a][i+b])
    max_value=np.max(values)
    return max_value

In [28]:
def indextolatlong(i,j):
    lat=top_left_x_coords[i]
    long=top_left_y_coords[j]
    return (lat,long)

In [29]:
def get_10googleimages(lon, lat):
    rootdir = '../Data/google_image/'
    i,j=get_cell_idx(lon, lat, top_left_x_coords, top_left_y_coords)
    print(i,j)
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if file == str(i) + '_' + str(j) +'.jpg':
                print("found")
                shutil.copy2(os.path.join(subdir, file),'../Data/google_image_clusters/')

In [30]:
list_i=[]
list_j=[]
for index, row in dataset.iterrows():
    i,j=utils.get_cell_idx(row['gpsLongitude'],row['gpsLatitude'], top_left_x_coords, top_left_y_coords)
    list_i.append(i)
    list_j.append(j)

In [43]:
from retrying import retry

@retry(wait_exponential_multiplier=1000, wait_exponential_max=3600000)
def save_img(url, file_path, file_name):
    a = urllib.urlopen(url).read()
    b = cStringIO.StringIO(a)
    image = ndimage.imread(b, mode='RGB')
    if np.array_equal(image[:,:10,:],image[:,10:20,:]):
        print("bad image")
    else:
        misc.imsave(file_path + file_name, image[50:450, :, :])

# 1 image per cluster

In [ ]:
m = 1
for i,j in zip(list_i,list_j):
        lon = centroid_x_coords[i]
        lat = centroid_y_coords[j]
        url = 'https://maps.googleapis.com/maps/api/staticmap?center=' + str(lat) + ',' + \
               str(lon) + '&zoom=16&size=400x500&maptype=satellite&key=' + key
        lightness = bands_data[j, i, 0]
        file_path = output_image_dir + str(lightness) + '/'
        if not os.path.isdir(file_path):
            os.makedirs(file_path)
        file_name = str(i) + '_' + str(j) +'.jpg'
        save_img(url, file_path, file_name)
        if m % 100 == 0:
            print m
        m += 1

# 100 images per cluster

In [44]:
m = 1
for i,j in zip(list_i,list_j):
    file_path = output_image_dir + str(i) + '_' + str(j) + '/'
    if not os.path.isdir(file_path):
        os.makedirs(file_path)
    for a in range(-5,5):
        for b in range(-5,5):
            lon = centroid_x_coords[i+a]
            lat = centroid_y_coords[j+b]
            url = 'https://maps.googleapis.com/maps/api/staticmap?center=' + str(lat) + ',' + \
                   str(lon) + '&zoom=16&size=400x500&maptype=satellite&key=' + key
            file_name = str(i+a) + '_' + str(j+b) +'.jpg'
            save_img(url, file_path, file_name)
            if m % 100 == 0:
                print m
            m += 1

100
200
300
400
500
600
700
800
900


/Users/pasquierjb/anaconda/lib/python2.7/site-packages/PIL/Image.py:870: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
1910